# Car price modeling with snowpark

## setup your local python development environment for snowpark


https://docs.snowflake.com/en/developer-guide/snowpark/python/setup

## setup connection to snowflake

Apply for a snowflake trial, .....
Make a note of the username password and accountname
enable Anoconda in the Admin > Billing & Terms section

create a python file connection_config.py with the following contents

```python
connection_parameters = {
    "account": "TLAABMO-GU51434", 
    "user": "snowflaketrialuser",
    "password": "Yourpassword_123",
    "warehouse": "COMPUTE_WH",
    "role": "accountadmin",
    "database": "SNOWFLAKE_SAMPLE_DATA",
    "schema": "TPCH_SF10"
}
```


In [3]:
connection_parameters

{'account': 'VBKEJIQ-KD94712',
 'user': 'snowflaketrialuser',
 'password': 'Yourpassword!0',
 'warehouse': 'COMPUTE_WH',
 'role': 'ACCOUNTADMIN',
 'database': 'SNOWFLAKE_SAMPLE_DATA',
 'schema': 'TPCH_SF10'}

In [1]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from connection_config import connection_parameters

import pandas as pd

#### Current Environment Details
def current_snowflake_env():
    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    print('User                     : {}'.format(snowflake_environment[0][0]))
    print('Role                     : {}'.format(snowflake_environment[0][1]))
    print('Database                 : {}'.format(snowflake_environment[0][2]))
    print('Schema                   : {}'.format(snowflake_environment[0][3]))
    print('Warehouse                : {}'.format(snowflake_environment[0][5]))
    print('Snowflake version        : {}'.format(snowflake_environment[0][4]))


In [2]:
connection_parameters

{'account': 'TLAABMO-GU51434',
 'user': 'snowflaketrialuser',
 'password': 'Yourpassword_123',
 'warehouse': 'COMPUTE_WH',
 'role': 'accountadmin',
 'database': 'SNOWFLAKE_SAMPLE_DATA',
 'schema': 'TPCH_SF10'}

In [3]:

#### Set up a connection with Snowflake
session = Session.builder.configs(connection_parameters).create()


In [4]:
current_snowflake_env()

User                     : SNOWFLAKETRIALUSER
Role                     : ACCOUNTADMIN
Database                 : SNOWFLAKE_SAMPLE_DATA
Schema                   : TPCH_SF10
Warehouse                : COMPUTE_WH
Snowflake version        : 7.34.1


In [5]:
session.add_packages("snowflake-snowpark-python", "pandas", "xgboost==1.7.3")

The version of package xgboost in the local environment is 1.7.6, which does not fit the criteria for the requirement xgboost==1.7.3. Your UDF might not work when the package version is different between the server and your local environment


## setup a new database

In [6]:
session.sql('CREATE OR REPLACE database cars_data').collect()


[Row(status='Database CARS_DATA successfully created.')]

In [7]:
session.sql('USE SCHEMA cars_data.public').collect()

[Row(status='Statement executed successfully.')]

## Get the cars data

from different cars sites we scraped cars for sale data, for each car we have....

In [8]:
car_prices = pd.read_csv("https://raw.githubusercontent.com/longhowlam/snowpark_cars_model/master/autos_tekoop.zip", encoding = "ISO-8859-1")

In [12]:
### extract number form vermogen column
car_prices['power'] = car_prices['vermogen'].str.extract('(\d+)')
car_prices['motor_inhoud'] = car_prices['motorinhoud'].str.extract('(\d+)')
car_prices['power'] = car_prices['power'].astype(float)
car_prices['motor_inhoud'] = car_prices['motor_inhoud'].astype(float)

In [13]:
car_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231000 entries, 0 to 230999
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   bouwjaar      231000 non-null  int64  
 1   km_stand      231000 non-null  int64  
 2   brandstof     231000 non-null  object 
 3   motorinhoud   217481 non-null  object 
 4   vermogen      225694 non-null  object 
 5   transmissie   230958 non-null  object 
 6   type          230469 non-null  object 
 7   kleur         228431 non-null  object 
 8   deur          228287 non-null  object 
 9   prijs         231000 non-null  object 
 10  merk          230996 non-null  object 
 11  model         231000 non-null  object 
 12  vraagprijs    231000 non-null  int64  
 13  power         216400 non-null  float64
 14  motor_inhoud  215971 non-null  float64
dtypes: float64(2), int64(3), object(10)
memory usage: 26.4+ MB


In [14]:
display(car_prices.sample(7))

,bouwjaar,km_stand,brandstof,motorinhoud,vermogen,transmissie,type,kleur,deur,prijs,merk,model,vraagprijs,power,motor_inhoud
210338,2005,107980,Benzine,1108cc,40kW,Handgeschakeld,Hatchback,Zwart,5-deurs,â¬ 2.250,Fiat,Panda,2250,40.0,1108.0
96384,2020,25000,Diesel,1496cc,90kW,Handgeschakeld,Stationwagon,Blauw,5-deurs,â¬ 32.390,Opel,Insignia,32390,90.0,1496.0
116578,2016,85902,Hybride,1969cc,235kW,Automaat,SUV / Terreinwagen,Grijs,5-deurs,â¬ 53.445,Volvo,XC90,53445,235.0,1969.0
109071,2022,0,Benzine,999cc,114kW,Handgeschakeld,SUV / Terreinwagen,5-deurs,5-deurs,â¬ 32.885,Ford,Puma,32885,114.0,999.0
173936,2011,115411,Benzine,1197cc,77kW,Handgeschakeld,Hatchback,Grijs,5-deurs,â¬ 7.997,Volkswagen,Polo,7997,77.0,1197.0
87038,2021,10,Hybride,1798cc,Automaat,Stationwagon,Blauw,5-deurs,5-deurs,â¬ 32.745,Suzuki,Swace,32745,NaN,1798.0
106357,2019,13303,Benzine,1498cc,109kW,Automaat,SUV / Terreinwagen,Wit,5-deurs,â¬ 29.875,Volkswagen,T-Roc,29875,109.0,1498.0


## create a snowflake table

In [15]:
## quote_identifiers set to False, 
## identifiers are passed on to Snowflake without quoting, i.e. identifiers will be coerced to uppercase by Snowflake.

session.write_pandas(car_prices, "CAR_PRICES", auto_create_table = True, quote_identifiers = False, overwrite = True)

## prepare data using snowpark
Now that we have a table in snowflake we are not using pandas to do data manipulation, but using snbowpark instead

In [16]:
cars_sf = session.table('CARS_DATA.PUBLIC.CAR_PRICES')

In [17]:
cars_sf.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BOUWJAAR"  |"KM_STAND"  |"BRANDSTOF"  |"MOTORINHOUD"  |"VERMOGEN"  |"TRANSMISSIE"  |"TYPE"               |"KLEUR"  |"DEUR"    |"PRIJS"     |"MERK"      |"MODEL"  |"VRAAGPRIJS"  |"POWER"  |"MOTOR_INHOUD"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2018        |54700       |Elektrisch   |NULL           | 245kW      |Automaat       |Hatchback            | Rood    | 5-deurs  |â¬ 54.999  |Tesla       |Model    |54999         |245.0    |NULL            |
|2017        |56266       |Elektrisch   |NULL           |NULL        |Automaat       | Hatchback           |Wit      | 5-deurs  |â¬ 22.949  |Volkswagen  |e-Golf   |229

### create new column age from bouwjaar

In [18]:
cars_sf = (
    cars_sf
    .with_column('age' , 2023 - cars_sf['BOUWJAAR'])
    .with_column('N_doors', cars_sf["DEUR"].substring(1,2))
)

In [19]:
cars_sf.sample(n=10).show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BOUWJAAR"  |"KM_STAND"  |"BRANDSTOF"  |"MOTORINHOUD"  |"VERMOGEN"  |"TRANSMISSIE"   |"TYPE"               |"KLEUR"  |"DEUR"    |"PRIJS"      |"MERK"    |"MODEL"   |"VRAAGPRIJS"  |"POWER"  |"MOTOR_INHOUD"  |"AGE"  |"N_DOORS"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2021        |10          |Hybride      | 1998cc        | 150kW      |Automaat        | Stationwagon        | Zwart   | 5-deurs  |â¬ 71.997   |BMW       |5-serie   |71997         |150.0    |1998.0          |2      | 5         |
|2022        |5           |Benzine      | 1496cc        | 88kW       |Handgeschakeld

In [20]:
cars_sf.count()

231000

### remove outliers

In [25]:
cars_clean = (
    cars_sf
    .filter(F.col("KM_STAND") <= 500000)
    .filter(F.col("AGE") <= 20 )
    .filter(F.col("TRANSMISSIE").in_(F.lit("Handgeschakeld"), F.lit("Automaat")) )
    .filter(F.col("VRAAGPRIJS") <= 100000)
    .filter(F.col("BRANDSTOF").in_(F.lit("Benzine"), F.lit("Diesel")) )
)

In [27]:
## drop the columns that we don't need
cars_clean = cars_clean.drop(["PRIJS","MOTORINHOUD", "VERMOGEN"] )

In [28]:
cars_clean.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"BOUWJAAR"  |"KM_STAND"  |"BRANDSTOF"  |"TRANSMISSIE"   |"TYPE"               |"KLEUR"  |"DEUR"    |"MERK"      |"MODEL"   |"VRAAGPRIJS"  |"POWER"  |"MOTOR_INHOUD"  |"AGE"  |"N_DOORS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2020        |7281        |Benzine      |Automaat        | SUV / Terreinwagen  | Rood    | 5-deurs  |CitroÃ«n    |C3        |26950         |96.0     |1199.0          |3      | 5         |
|2015        |26120       |Benzine      |Handgeschakeld  | Hatchback           | Wit     | 3-deurs  |Fiat        |500       |9750          |51.0     |1242.0          |8      | 3         |
|2018        |44398       |Benzine      |Handgeschakeld  | S

### save the data into a snowflake table

In [29]:
cars_clean.count()

179490

In [30]:
cars_clean.write.mode("overwrite").save_as_table("CARS_DATA.PUBLIC.CARS_CLEAN")

## Gracefully close snowflake session

In [31]:
session.close()